Conversion to grayscale videos.

In [1]:
import cv2
import os

def convert_video_to_grayscale(input_video_path, output_video_path):
    # Open the input video
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video {input_video_path}")
        return

    # Get the video properties (width, height, fps)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    print(f"Processing video: {input_video_path}")
    print(f"Video resolution: {frame_width}x{frame_height}")
    print(f"Video FPS: {fps}")
    
    # Determine the output video format (keeping the same format as input)
    file_extension = os.path.splitext(output_video_path)[1].lower()

    # Set codec based on the extension
    if file_extension == '.mp4':
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' for .mp4 format
    elif file_extension == '.mov':
        fourcc = cv2.VideoWriter_fourcc(*'avc1')  # 'avc1' is common for .mov files (H.264 codec)
    else:
        print(f"Error: Unsupported video format. Supported formats are .mp4 and .mov.")
        cap.release()
        return

    # Set up the video writer for the output file (grayscale video)
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height), isColor=False)

    # Read and process each frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Write the grayscale frame to the output video
        out.write(gray_frame)

    # Release resources
    cap.release()
    out.release()
    print(f"Grayscale video saved as {output_video_path}.")

def process_videos_in_folder(input_folder_path):
    # Check if the directory exists
    if not os.path.exists(input_folder_path):
        print(f"Error: Folder {input_folder_path} does not exist.")
        return
    
    # Loop through all files in the folder
    for filename in os.listdir(input_folder_path):
        # Get full file path
        input_video_path = os.path.join(input_folder_path, filename)
        
        # Check if the file is a video (.mp4 or .mov)
        if filename.lower().endswith(('.mp4', '.mov')):
            # Prepare the output path (same folder, with '_grayscale' appended)
            output_video_path = input_video_path #.replace(os.path.splitext(filename)[1], '_grayscale' + os.path.splitext(filename)[1])
            
            # Convert the video to grayscale
            convert_video_to_grayscale(input_video_path, output_video_path)
        else:
            print(f"Skipping {filename}: Not a video file.")

if __name__ == "__main__":
    # Set the folder path where the videos are located
    input_folder_path = "../dataset_clips"  # Replace with the actual folder path

    # Process all videos in the folder
    process_videos_in_folder(input_folder_path)

Processing video: ../dataset_clips\april_0.MOV
Video resolution: 640x360
Video FPS: 60.0
Grayscale video saved as ../dataset_clips\april_0.MOV.
Processing video: ../dataset_clips\april_1.MOV
Video resolution: 640x360
Video FPS: 60.0
Grayscale video saved as ../dataset_clips\april_1.MOV.
Processing video: ../dataset_clips\april_10.MOV
Video resolution: 640x360
Video FPS: 60.0
Grayscale video saved as ../dataset_clips\april_10.MOV.
Processing video: ../dataset_clips\april_11.MOV
Video resolution: 640x360
Video FPS: 60.0
Grayscale video saved as ../dataset_clips\april_11.MOV.
Processing video: ../dataset_clips\april_12.MOV
Video resolution: 640x360
Video FPS: 60.0
Grayscale video saved as ../dataset_clips\april_12.MOV.
Processing video: ../dataset_clips\april_13.MOV
Video resolution: 640x360
Video FPS: 60.0
Grayscale video saved as ../dataset_clips\april_13.MOV.
Processing video: ../dataset_clips\april_14.MOV
Video resolution: 640x360
Video FPS: 60.0
Grayscale video saved as ../dataset_cl

Split to 80% train and 20% test set (per class)

In [2]:
import os
import random
import shutil

def get_class_from_filename(filename):
    return filename.rsplit('_', 1)[0]  # Splits from the right, keeping everything before the last '_'

def split_video_files_by_class(input_folder, train_folder, test_folder, train_ratio=0.8):
    # Check if the input folder exists
    if not os.path.exists(input_folder):
        print(f"Error: The input folder {input_folder} does not exist.")
        return

    # Create the train and test folders if they don't exist
    if not os.path.exists(train_folder):
        os.makedirs(train_folder)
    if not os.path.exists(test_folder):
        os.makedirs(test_folder)

    # List all files in the input folder
    all_files = os.listdir(input_folder)

    # Filter only video files (e.g., .mp4, .mov, etc.)
    video_extensions = ('.mp4', '.mov')
    video_files = [f for f in all_files if f.lower().endswith(video_extensions)]

    # Group files by their class (assuming class is part of the filename)
    class_groups = {}
    for file in video_files:
        class_name = get_class_from_filename(file)
        if class_name not in class_groups:
            class_groups[class_name] = []
        class_groups[class_name].append(file)

    # Loop through each class and split its files into train/test
    for class_name, files in class_groups.items():
        # Shuffle the list of files in this class
        random.shuffle(files)

        # Calculate the number of training and test videos for this class
        class_size = len(files)
        train_size = int(class_size * train_ratio)
        
        # Split the files into train and test sets
        train_files = files[:train_size]
        test_files = files[train_size:]

        # Copy files to the train folder
        for file in train_files:
            src = os.path.join(input_folder, file)
            dst = os.path.join(train_folder, file)
            shutil.copy(src, dst)  # Use copy instead of move
            print(f"Copied {file} to {train_folder}")

        # Copy files to the test folder
        for file in test_files:
            src = os.path.join(input_folder, file)
            dst = os.path.join(test_folder, file)
            shutil.copy(src, dst)  # Use copy instead of move
            print(f"Copied {file} to {test_folder}")

if __name__ == "__main__":
    # Path to your dataset folder containing video files
    input_folder = "../dataset_clips"  # Replace with the actual path

    # Path where the train and test folders should be created
    train_folder = os.path.join(input_folder, "../model_input/train_clips")
    test_folder = os.path.join(input_folder, "../model_input/test_clips")

    # Call the split function
    split_video_files_by_class(input_folder, train_folder, test_folder)

Copied april_15.MOV to ../dataset_clips\../model_input/train_clips
Copied april_1.MOV to ../dataset_clips\../model_input/train_clips
Copied april_3.MOV to ../dataset_clips\../model_input/train_clips
Copied april_5.MOV to ../dataset_clips\../model_input/train_clips
Copied april_14.MOV to ../dataset_clips\../model_input/train_clips
Copied april_13.MOV to ../dataset_clips\../model_input/train_clips
Copied april_2.MOV to ../dataset_clips\../model_input/train_clips
Copied april_7.MOV to ../dataset_clips\../model_input/train_clips
Copied april_12.MOV to ../dataset_clips\../model_input/train_clips
Copied april_16.MOV to ../dataset_clips\../model_input/train_clips
Copied april_11.MOV to ../dataset_clips\../model_input/train_clips
Copied april_0.MOV to ../dataset_clips\../model_input/train_clips
Copied april_17.MOV to ../dataset_clips\../model_input/train_clips
Copied april_19.MOV to ../dataset_clips\../model_input/train_clips
Copied april_4.MOV to ../dataset_clips\../model_input/train_clips
Co